# Tutorial: Mounting NVMe SSD in EC2 Instances

## Step 1: List Available NVMe Devices
List the available NVMe devices using the command:
```bash
ls /dev/nvme*
```

## Step 2: Identify the NVMe Device Using `lsblk`
```bash
lsblk
```
The `lsblk` command will list all the block devices attached to your instance, showing their mount points, size, and other relevant details. Here’s an example of what the output might look like:
```
NAME        MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT
nvme0n1     259:0    0   65G  0 disk
└─nvme0n1p1 259:1    0   65G  0 part /
nvme1n1     259:2    0  225G  0 disk
```
In this example, `nvme0n1` is the root EBS volume (65 GiB) mounted on `/`, and `nvme1n1` is the additional NVMe SSD instance store volume (225 GiB).


## Step 3: Check Mounted File Systems
Run the following command to check the mounted file systems:
```bash
df -h
```
Example output:
```
Filesystem      Size  Used Avail Use% Mounted on
/dev/root        63G   48G   16G  77% /
tmpfs            32G     0   32G   0% /dev/shm
tmpfs            13G  980K   13G   1% /run
tmpfs           5.0M     0  5.0M   0% /run/lock
efivarfs        128K  4.1K  119K   4% /sys/firmware/efi/efivars
/dev/nvme0n1p15 105M  6.1M   99M   6% /boot/efi
/dev/mapper/vg.01-lv_ephemeral 206G 28K 195G 1% /opt/dlami/nvme
tmpfs           6.3G  4.0K  6.3G   1% /run/user/1000
```


## Step 4: Verify the NVMe SSD Volume and Mount Point
Example `lsblk` output:
```
NAME                 MAJ:MIN RM  SIZE RO TYPE MOUNTPOINTS
loop0                  7:0    0  25.2M  1 loop /snap/amazon-ssm-agent/7993
loop1                  7:1    0  55.7M  1 loop /snap/core18/2829
loop2                  7:2    0  63.9M  1 loop /snap/core20/2318
loop3                  7:3    0  87M    1 loop /snap/lxd/28373
loop4                  7:4    0  38.8M  1 loop /snap/snapd/21759
loop5                  7:5    0  87M    1 loop /snap/lxd/29351
nvme0n1              259:0    0   65G  0 disk 
├─nvme0n1p1          259:2    0 64.9G  0 part /
├─nvme0n1p14         259:3    0    4M  0 part 
└─nvme0n1p15         259:4    0  106M  0 part /boot/efi
nvme1n1              259:1    0 209.5G  0 disk 
└─vg.01-lv_ephemeral 252:0    0 209.5G  0 lvm  /opt/dlami/nvme
```


## Step 5: Remount the Volume to a Different Directory
If you want to mount the volume to a different directory, follow these steps:
1. Unmount the current mount point:
    ```bash
    sudo umount /opt/dlami/nvme
    ```
2. Create a new directory for the mount point (if it doesn’t already exist):
    ```bash
    sudo mkdir /data
    ```
3. Mount the NVMe device to the new directory:
    ```bash
    sudo mount /dev/mapper/vg.01-lv_ephemeral /data
    ```
4. Verify the new mount:
    ```bash
    df -h
    ```


## Step 6: Automate the Mounting Process
To ensure the volume is mounted to `/data` on every reboot, add an entry to the `/etc/fstab` file:
1. Open `/etc/fstab` in a text editor:
    ```bash
    sudo nano /etc/fstab
    ```
2. Add the following line to the end of the file (replace `ext4` with the actual file system type if it is different):
    ```bash
    /dev/mapper/vg.01-lv_ephemeral /data ext4 defaults,nofail 0 2
    ```
3. Save and close the file.
4. Test the `/etc/fstab` entry by unmounting and remounting:
    ```bash
    sudo umount /data
    sudo mount -a
    ```


## Step 7: Determine the File System Type
To determine the file system type of your NVMe SSD instance store volume:
```bash
lsblk -f
```
Example output:
```
NAME                 FSTYPE      LABEL UUID                                   MOUNTPOINT
nvme0n1                                                                                       
├─nvme0n1p1         ext4              e3b9f8a0-2d24-4c1d-a2d8-7c62b6b539c1   /
└─nvme0n1p15        vfat              70CB-DA4A                              /boot/efi
nvme1n1             ext4              a1c9d634-4e13-4fd1-bddc-09c8de8d1d65   /opt/dlami/nvme
```


## Step 8: Access the Mount Point from Home Directory
### Method 1: Creating a Symlink
1. Create a symlink:
    ```bash
    ln -s /data /home/ubuntu/data
    ```
2. Access the symlink:
    ```bash
    cd /home/ubuntu/data
    ```

### Method 2: Bind Mount
1. Create a new directory in your home directory:
    ```bash
    mkdir /home/ubuntu/data
    ```
2. Bind mount the directory:
    ```bash
    sudo mount --bind /data /home/ubuntu/data
    ```
3. Verify the bind mount:
    ```bash
    df -h
    ```
4. Access the bind mount:
    ```bash
    cd /home/ubuntu/data
    ```

### Method 3: Editing `/etc/fstab` for Persistent Bind Mount
1. Open `/etc/fstab`:
    ```bash
    sudo nano /etc/fstab
    ```
2. Add the following line:
    ```bash
    /data /home/ubuntu/data none bind 0 0
    ```
3. Save and close the file (Ctrl+O, Enter, Ctrl+X).
4. Test the `/etc/fstab` entry:
    ```bash
    sudo mount -a
    ```
5. Verify the bind mount:
    ```bash
    df -h
    ```


## Summary
By following these steps, you can successfully mount your NVMe SSD instance store volume to a desired directory and access it from your home directory, ensuring easy access and persistent configuration.
